In [15]:
from spire.doc import *
from spire.doc.common import *
import os


from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# imports
import ast  # for converting embeddings saved as strings back to arrays
from openai import OpenAI # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
 # for getting API token from env variable OPENAI_API_KEY
from scipy import spatial  # for calculating vector similarities for search

In [16]:
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
 
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

EMBEDDING_MODEL = "text-embedding-3-small"
GPT_MODEL = "gpt-3.5-turbo"

In [17]:
def get_embedding(text, model=EMBEDDING_MODEL):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

### GENERACIÓN TABLA


In [18]:
import pandas as pd

In [19]:
df = pd.read_excel("DATOS LIMPIOS IASENS PARTE 1 - SOLO NACIONALIDAD ESPAÑA.xlsx", engine='openpyxl')

In [20]:
text_table = df[['TEXTO']]

In [21]:
categories_table = pd.DataFrame(["GENERAL", "ASPECTO", "OLOR", "TEXTURA", "SABOR", "COLOR", "DUREZA", "DULZOR"], columns=["categorias"])

In [22]:
from tqdm.auto import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

In [23]:
categories_table['embeddings'] = categories_table['categorias'].apply(lambda x: get_embedding(x, model=EMBEDDING_MODEL))

In [48]:
categories_table.to_csv("tabla_de_categorias")

### ENCONTRAR QUÉ CATEGORIAS FALTAN EN EL TEXTO PROPORCIONADO

In [24]:
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [30]:
categories_table

,categorias,embeddings
0,GENERAL,"[0.03326110914349556, 0.03465009108185768, 0.0..."
1,ASPECTO,"[-0.0074445027858018875, 0.008860873989760876,..."
2,OLOR,"[-0.0061168791726231575, -0.024890098720788956..."
3,TEXTURA,"[0.008095785975456238, 0.014355413615703583, -..."
4,SABOR,"[0.032947197556495667, 0.01949639990925789, -0..."
5,COLOR,"[0.013108697719871998, -0.018466662615537643, ..."
6,DUREZA,"[0.04361307621002197, 0.02947567217051983, 0.0..."
7,DULZOR,"[0.04333304241299629, 0.02525906264781952, -0...."


In [40]:
def get_similarity_table(text, categories = categories_table):
    similarity_table = categories[['categorias']]
    embedded_text = get_embedding(text, model=EMBEDDING_MODEL)
    similarity_table['similitud'] = categories.embeddings.apply(lambda x: cosine_similarity(x, embedded_text))
    return similarity_table

In [46]:
text_table['TEXTO'][0]

'apariencia divertida perfecta para un público infantil.Me gusta el color tostadoy el sabor es bueno. Aunque es muy pegajosa en el paladar'

In [43]:
ranked_table = get_similarity_table(text_table['TEXTO'][0]).sort_values(by="similitud", ascending=True)

  categorias                                         embeddings  similitud
0    GENERAL  [0.03326110914349556, 0.03465009108185768, 0.0...   0.082019
1    ASPECTO  [-0.0074445027858018875, 0.008860873989760876,...   0.261545
2       OLOR  [-0.0061168791726231575, -0.024890098720788956...   0.180538
3    TEXTURA  [0.008095785975456238, 0.014355413615703583, -...   0.297098
4      SABOR  [0.032947197556495667, 0.01949639990925789, -0...   0.410541


In [62]:
def get_categories(similarities):
    similarities.sort_values(by="similitud", ascending=True)
    categories = similarities[similarities["similitud"] < 0.2]["categorias"].tolist()
    return categories

In [71]:
lista = get_categories(ranked_table)
lista

['GENERAL', 'DULZOR', 'OLOR']

In [73]:
lista.pop(0)

'GENERAL'

In [74]:
lista

['DULZOR', 'OLOR']

In [49]:
PLANTILLAS_PREGUNTAS = ["General: ¿Puedes contarme más en general?",
"Aspecto: ¿Cómo describirías su aspecto?",
"Olor: ¿Qué me puedes decir sobre su olor?",
"Textura: ¿Cómo describirías su textura?",
"Sabor: ¿Qué me puedes decir sobre su sabor?",
"Color: ¿Cómo describirías su color?",
"Dureza: ¿Qué tan duro es?",
"Dulzura: ¿Puedes comentar sobre su nivel de dulzura?"]

In [ ]:
def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    res = get_similarity_table( query)
    strings = get_categories(res)
    introduction = 'Utiliza los campos de abajo para generar UNICAMENTE preguntas . Si la pregunta no puede ser encontrada en los artículos o sobre el contexto de la conversación, escribe "No he podido encontrar la respuesta"'
    question = f"\n\Pregunta: {query} "
    conversation_context = f"Contexto de la conversación: {get_conversation_context()}"
    message = introduction
    for string in strings:
        next_article = f'\n\nArtículos:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question + conversation_context, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question + conversation_context


In [58]:
for elem in PLANTILLAS_PREGUNTAS:
    print(elem)

TypeError: 'str' object is not callable